In [3]:
import os
import json
import pandas as pd

data_folder = "/Users/abhishekkumar/Desktop/data_quality_project/data1"
all_files = [f for f in os.listdir(data_folder) if f.endswith(('.json', '.jsonl'))]

dfs = {}
for f in all_files:
    file_path = os.path.join(data_folder, f)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]
    df = pd.json_normalize(data)
    df_dedup = df.astype(str).drop_duplicates()  # deduplicate safely
    dfs[f] = df_dedup

print(f"Loaded {len(dfs)} DataFrames.")

Loaded 24 DataFrames.


In [14]:
dq_df.head()

,file,total_missing,missing_per_column,duplicate_rows
0,news_events_2025_07_07.00086.jsonl,0,"{'data': 0, 'included': 0}",0
1,news_events_2025_07_07.00003.jsonl,0,"{'data': 0, 'included': 0}",0
2,news_events_2025_07_07.00001.jsonl,0,"{'data': 0, 'included': 0}",0
3,news_events_2025_07_07.00084.jsonl,0,"{'data': 0, 'included': 0}",0
4,news_events_2025_07_07.00005.jsonl,0,"{'data': 0, 'included': 0}",0


In [4]:
dq_dimensions = ['Accuracy', 'Completeness', 'Consistency', 'Uniqueness', 'Validity', 'Timeliness']

In [5]:
dq_summary = []

for file_name, df in dfs.items():
    dq_report = {'file': file_name}

    # Completeness
    dq_report['total_missing'] = df.isnull().sum().sum()
    dq_report['missing_per_column'] = df.isnull().sum().to_dict()

    # Uniqueness
    dq_report['duplicate_rows'] = df.duplicated().sum()

    # Consistency example: cross-field check
    # E.g., 'event_date' <= today
    if 'event_date' in df.columns:
        dq_report['future_dates'] = (pd.to_datetime(df['event_date'], errors='coerce') > pd.Timestamp.today()).sum()
    
    # Accuracy / Validity example: regex for headlines
    if 'headline' in df.columns:
        dq_report['headline_invalid'] = df['headline'].apply(lambda x: not bool(str(x).strip()) if pd.notnull(x) else True).sum()
    
    dq_summary.append(dq_report)